# Imagen: Dogs vs Cats dataset

In [4]:
import sys
!{sys.executable} -m pip install tensorflow --quiet



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import os, zipfile
import tensorflow as tf
import pickle


In [6]:
import zipfile
import pathlib

zip_path = "deep-learning.zip"
extract_path = "deep-learning"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

BASE_DIR   = pathlib.Path("deep-learning/deep-learning")
IMG_SIZE   = (224, 224)
BATCH_SIZE = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
              BASE_DIR / "train",
              validation_split=0.2, subset="training", seed=123,
              image_size=IMG_SIZE, batch_size=BATCH_SIZE,
              label_mode="binary")

val_ds   = tf.keras.preprocessing.image_dataset_from_directory(
              BASE_DIR / "train",
              validation_split=0.2, subset="validation", seed=123,
              image_size=IMG_SIZE, batch_size=BATCH_SIZE,
              label_mode="binary")

test_ds  = tf.keras.preprocessing.image_dataset_from_directory(
              BASE_DIR / "test",
              shuffle=False,
              image_size=IMG_SIZE, batch_size=BATCH_SIZE,
              label_mode="binary")

print("Clases detectadas:", train_ds.class_names)


Found 557 files belonging to 2 classes.
Using 446 files for training.
Found 557 files belonging to 2 classes.
Using 111 files for validation.
Found 140 files belonging to 2 classes.
Clases detectadas: ['cats', 'dogs']


In [7]:
!pip install tensorflow.keras

In [8]:
from tensorflow import keras
from tensorflow.keras import layers


image_model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

image_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


image_model.fit(train_ds,
          epochs=15,
          validation_data=val_ds)

test_loss, test_acc = image_model.evaluate(test_ds)
print("Test accuracy:", test_acc)


Epoch 1/15


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.5051 - loss: 37.3107 - val_accuracy: 0.5135 - val_loss: 10.0036
Epoch 2/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.5604 - loss: 10.9630 - val_accuracy: 0.4955 - val_loss: 10.8025
Epoch 3/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.5263 - loss: 11.1192 - val_accuracy: 0.4865 - val_loss: 15.8541
Epoch 4/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.5785 - loss: 8.3926 - val_accuracy: 0.4865 - val_loss: 5.6655
Epoch 5/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.5909 - loss: 7.4874 - val_accuracy: 0.5045 - val_loss: 8.7056
Epoch 6/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.5501 - loss: 15.1269 - val_accuracy: 0.4865 - val_loss: 15.4878
Epoch 7/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.5320 - loss: 9.3946 - val_accuracy: 0.5495 - val_loss: 6.5247
Epoch 8/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.7061 - loss: 2.8386 - val_accuracy: 0.5676 - val_

In [9]:
import sys
!{sys.executable} -m pip install scikit-learn --quiet


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [10]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
y_true = np.concatenate([y for x, y in test_ds], axis=0)

y_pred_prob = image_model.predict(test_ds)
y_pred = (y_pred_prob > 0.5).astype("int32").flatten()
print(classification_report(y_true, y_pred, target_names=test_ds.class_names))


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
              precision    recall  f1-score   support

        cats       1.00      0.06      0.11        70
        dogs       0.51      1.00      0.68        70

    accuracy                           0.53       140
   macro avg       0.76      0.53      0.39       140
weighted avg       0.76      0.53      0.39       140



2025-05-28 11:36:01.830430: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:
image_model.save("image_model.keras")


# Text SMS Spam Collection

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [13]:
!{sys.executable} -m pip install pandas --quiet


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [14]:
import pandas as pd
df = pd.read_csv("spam.csv", encoding="latin-1")[['v1', 'v2']]
df = df.rename(columns={'v1': 'label', 'v2': 'text'}).copy()
text_model = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', MultinomialNB())
])

In [15]:
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [16]:

df['label'] = df['label'].str.strip().map({'ham': 0, 'spam': 1}).astype('int32')
df['text']  = df['text'].astype(str)

In [17]:
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GlobalAveragePooling1D, Dense


X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42)

MAX_TOKENS = 10000
MAX_LEN = 100
EMBED_DIM = 16

vectorizer = TextVectorization(max_tokens=MAX_TOKENS,
                               output_sequence_length=MAX_LEN)
vectorizer.adapt(X_train)

text_model = Sequential([
    vectorizer,
    Embedding(MAX_TOKENS, EMBED_DIM),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

text_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

X_train_np = X_train.to_numpy(dtype=object)
X_test_np  = X_test.to_numpy(dtype=object)

text_model.fit(X_train_np, y_train.to_numpy(),
          epochs=15, batch_size=32,
          validation_split=0.1)

loss, acc = text_model.evaluate(X_test_np, y_test.to_numpy())
print(f"Test accuracy: {acc:.3f}")

Epoch 1/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8714 - loss: 0.4563 - val_accuracy: 0.8565 - val_loss: 0.3906
Epoch 2/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 969us/step - accuracy: 0.8663 - loss: 0.3667 - val_accuracy: 0.8565 - val_loss: 0.3854
Epoch 3/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step - accuracy: 0.8733 - loss: 0.3448 - val_accuracy: 0.8565 - val_loss: 0.3708
Epoch 4/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 987us/step - accuracy: 0.8621 - loss: 0.3474 - val_accuracy: 0.8565 - val_loss: 0.3247
Epoch 5/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step - accuracy: 0.8765 - loss: 0.2630 - val_accuracy: 0.9170 - val_loss: 0.1985
Epoch 6/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 987us/step - accuracy: 0.9435 - loss: 0.1592 - val_accuracy: 0.9529 - val_loss: 0.1418
Epoch 7/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 972us/step - accuracy: 0.9722 - loss: 0.0917 - val_accuracy: 0.9641 - val_loss: 0.0995
Epoch 8/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 967us/step - accuracy: 0.9802 - loss: 0.0678 - va

In [18]:
from sklearn.metrics import classification_report

y_pred_probs = text_model.predict(X_test_np)

y_pred_labels = (y_pred_probs > 0.5).astype(int)

print(classification_report(y_test, y_pred_labels,
                            target_names=['ham', 'spam']))


35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       965
        spam       0.98      0.86      0.92       150

    accuracy                           0.98      1115
   macro avg       0.98      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [19]:
text_model.save("text_model.keras")

# Regression: House Prices

In [20]:
train = pd.read_csv("train.csv")

In [21]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [22]:
len(train)

1460

In [23]:
len(train.columns)

81

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from tensorflow.keras import layers, Sequential
import numpy as np

train = train.fillna(train.mean(numeric_only=True))

y = train.pop('SalePrice')
X = train.copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X_train.select_dtypes(exclude=['object', 'category']).columns.tolist()

pre = ColumnTransformer([
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ('num', MinMaxScaler(), num_cols)
    ])

X_train_prep = pre.fit_transform(X_train)
X_test_prep  = pre.transform(X_test)

 
y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1,1))

input_dim = X_train_prep.shape[1]

regression_model = Sequential([
    layers.Dense(128, activation='relu', input_shape=(input_dim,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

regression_model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

history = regression_model.fit(
    X_train_prep, y_train_scaled,
    epochs=50, batch_size=32,
    validation_split=0.1,
    verbose=1
)

y_pred_scaled = regression_model.predict(X_test_prep)
y_pred = y_scaler.inverse_transform(y_pred_scaled)

mae = np.mean(np.abs(y_test.values.reshape(-1,1) - y_pred))
print(f"Test MAE (original scale): {mae:.2f}")


Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0846 - mean_absolute_error: 0.2203 - val_loss: 0.0077 - val_mean_absolute_error: 0.0672
Epoch 2/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0071 - mean_absolute_error: 0.0630 - val_loss: 0.0058 - val_mean_absolute_error: 0.0579
Epoch 3/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0038 - mean_absolute_error: 0.0454 - val_loss: 0.0050 - val_mean_absolute_error: 0.0548
Epoch 4/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0026 - mean_absolute_error: 0.0379 - val_loss: 0.0046 - val_mean_absolute_error: 0.0530
Epoch 5/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0017 - mean_absolute_error: 0.0304 - val_loss: 0.0043 - val_mean_absolute_error: 0.0507
Epoch 6/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0014 - mean_absolute_error: 0.0281 - val_loss: 0.0042 - val_mean_absolute_error: 0.0505
Epoch 7/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.7686e-04 - mean_absolute_error: 0.0233 - val_loss: 0.0041 - val

In [25]:
loss, mae = regression_model.evaluate(X_test_prep, y_test)
print(f"Test MAE: {mae:.0f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38920503296.0000 - mean_absolute_error: 178000.3281 
Test MAE: 178840


In [26]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import math
y_pred_scaled = regression_model.predict(X_test_prep).squeeze()

y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).squeeze()

mae  = mean_absolute_error(y_test, y_pred)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2   = r2_score(y_test, y_pred)

print(f"MAE:  ${mae:,.0f}")
print(f"RMSE: ${rmse:,.0f}")
print(f"R² score: {r2:.3f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MAE:  $30,815
RMSE: $43,271
R² score: 0.756


In [27]:
regression_model.save("regression_model.keras")

In [28]:

with open("preprocessor.pkl", "wb") as f:
    pickle.dump(pre, f)

with open("y_scaler.pkl", "wb") as f:
    pickle.dump(y_scaler, f)


In [29]:
!pip freeze > requirements.txt
